# Enunciado de la Actividad Práctica


***A)*** Subir un video MP4 a google colab y crear un kernel de GPU en
donde se combine en el mismo frame los siguientes filtros:


*   ***Mitad superior:*** Inversion de colores
*   ***Mitad inferior:*** Escala de grises

El resultado final deberia ser un video en donde se pueda ver en donde se apliquen los 2 filtros al mismo tiempo en cada mitad del video

**B)** Utilice NVProf para medir la velocidad de respuesta que tiene el algoritmo durante su ejecución de acuerdo a la siguiente configuración y describa que sucede con los tiempos.

| Configuración de Bloque   | Tiempo de ejecución del kernel|
|---------------------------|-------------------------------|
| (8,8,1)                 |                               |
| (32,32,1)                 |                               |
| máximo soportado por GPU  |                               |



Tips: Para saber cual es el tamaño maximo de la configuración soportado por la GPU ejecute el siguiente comando:

In [ ]:
!rm -rf cuda-samples
!git clone https://github.com/NVIDIA/cuda-samples.git
! cd cuda-samples/Samples/1_Utilities/deviceQuery; nvcc deviceQuery.cpp -I ../../../Common -o deviceQuery
!echo "------------------------------------------------------------------- "
!cuda-samples/Samples/1_Utilities/deviceQuery/deviceQuery

Cloning into 'cuda-samples'...
remote: Enumerating objects: 30467, done.
remote: Total 30467 (delta 0), reused 0 (delta 0), pack-reused 30467 (from 1)
Receiving objects: 100% (30467/30467), 137.16 MiB | 15.60 MiB/s, done.
Resolving deltas: 100% (27016/27016), done.
Updating files: 100% (2052/2052), done.
------------------------------------------------------------------- 
cuda-samples/Samples/1_Utilities/deviceQuery/deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.4 / 12.5
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15095 MBytes (15828320256 bytes)
  (040) Multiprocessors, (064) CUDA Cores/MP:    2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                          

#1) Subir video MP4 a Colab
En esta sección el alumno debe subir el archivo de video con formato MP4 a colab para poder ser trabajado en la GPU


In [7]:
#codigo para subir el archivo a colab
from google.colab import files
import os

#Aca se abre el cuadro de dialogo para seleccionar el archivo mp4 a subir
uploaded = files.upload()

#Aca se obtiene el nombre subido por el alumno y se lo renombra a video.mp4
filename = next(iter(uploaded))
new_filename = "video.mp4"
os.rename(filename, new_filename)

print(f"El alumno ha subido el archivo de video a Colab con el nombre: {new_filename}")

Saving video.mp4 to video (1).mp4
El alumno ha subido el archivo de video a Colab con el nombre: video.mp4


# 2) Reproducción del video
En esta sección el alumno podrá ver el video en la misma pagina de colab

In [8]:
#codigo para reproducir el video subido por el alumno
from IPython.display import HTML
from base64 import b64encode

mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


# 3) Instalación de componentes para CUDA
En esta sección se instala los componentes necearios para hacer la aplicacion de los filtros con GPU, usando Pycuda y OPENCV

In [3]:
!apt-get install ffmpeg -y
!pip install pycuda opencv-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 11.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=64dd62c67e8769d26ba9b3cb01f50f0a9a6423997e6d4e56bd6d8ecd0e7ecf51
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda


# 4)Crear script python con pycuda
En esta sección se crea el programa python con pycuda para ejecutar en la GPU

In [4]:
%%writefile filter_in_video_mp4.py

import cv2
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import subprocess
import os
from google.colab import files

#Contsatente utilizadas para la conversion del video
INPUT_VIDEO_MP4 = "video.mp4"
TEMP_VIDEO = "output_.avi"
FINAL_VIDEO_MP4_GPU = "video_with_filter_gpu.mp4"

#constante utilizada para separar el audio
TEMP_AUDIO = "audio.aac"

kernel_code = """
__global__ void Kernel_GPU(unsigned char* inputImage, unsigned char* outputImage, int imageWidth, int imageHeight)
 {
    int pixelX = blockIdx.x * blockDim.x + threadIdx.x;
    int pixelY = blockIdx.y * blockDim.y + threadIdx.y;

    if (pixelX >= imageWidth || pixelY >= imageHeight)
        return;

    //******************************************************
    //*******************COMPLETAR POR LOS ALUMNOS**********
    //******************************************************

}
"""

def load_video():
    #se lee el video y se obtienen los fps
    cap = cv2.VideoCapture(INPUT_VIDEO_MP4)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    #se crea el video a color
    out = cv2.VideoWriter(TEMP_VIDEO,
                          cv2.VideoWriter_fourcc(*'XVID'),
                          fps, (width, height), True)
    return out,cap,height,width

def join_audio_with_video():
    print("Extrayendo audio...")
    subprocess.run(["ffmpeg", "-y", "-i", INPUT_VIDEO_MP4, "-q:a", "0", "-map", "a", TEMP_AUDIO])

    print("Combinando audio y video...")
    if os.path.exists(TEMP_AUDIO):
        # Si el audio existe, se combina con el video
        subprocess.run([
            "ffmpeg", "-y", "-i", TEMP_VIDEO, "-i", TEMP_AUDIO,
            "-c:v", "copy", "-c:a", "aac", "-strict", "experimental", FINAL_VIDEO_MP4_GPU
        ])
        os.remove(TEMP_AUDIO)
    else:
        # Si el audio no existe, solo se guarda el video
        print("El archivo de audio no existe. Se guarda solo el video.")
        subprocess.run([
            "ffmpeg", "-y", "-i", TEMP_VIDEO,
            "-c:v", "copy", FINAL_VIDEO_MP4_GPU
        ])


def main():
    #defino cual va a ser el kernel que se va a ejecutar en la gpu
    mod = SourceModule(kernel_code)
    process_frame = mod.get_function("Kernel_GPU")

    # Aca se llama al kernel para ser ejecutado en la GPU
    out,cap,height,width=load_video()

    #determino cual es el tamaño del frame
    frame_size_rgb = height * width * 3

    #se define los bloques y la grilla
    block = (32,32, 1)
    grid = ((width + block[0] - 1) // block[0],
        (height + block[1] - 1) // block[1], 1)

    #se reserva la memoria en la cpu inicializandola en ceros
    input_host = np.zeros(shape=(height, width, 3), dtype=np.uint8)
    output_host = np.zeros(shape=(height, width, 3), dtype=np.uint8)

    #me reserva memoria en la gpu
    input_gpu = cuda.mem_alloc(frame_size_rgb)
    output_gpu = cuda.mem_alloc(frame_size_rgb)

    #se hace el procesmaiento de los frames
    #mientras el objeto cap que controla el video este abierto lee frames por frame
    print("Procesando video...")

    while cap.isOpened():

      #se lee el siguiente frame
      ret, frame = cap.read()

      #si no pudo leer el frame, quiere decir que termino el video
      if not ret:
          break

      #copio el frame a la memoria de la cpu y a la gpu
      np.copyto(input_host, frame)
      cuda.memcpy_htod(input_gpu, input_host)

      #ejecuto al kernel en la gpu
      process_frame(input_gpu, output_gpu, np.int32(width), np.int32(height), block=block, grid=grid)

      #una vez que termine de ejecutar el kernel copio el resultado del frame de la GPU a la CPU
      cuda.memcpy_dtoh(output_host, output_gpu)
      out.write(output_host)

    cap.release()
    out.release()

    join_audio_with_video()

    print("El video final con sonido es:", FINAL_VIDEO_MP4_GPU)

    #Libero la memoria de la gpu
    input_gpu.free()
    output_gpu.free()

    # Limpiar archivos temporales
    os.remove(TEMP_VIDEO)

main()

Writing filter_in_video_mp4.py


# 5) Ejecutar el script de python generado usando pycuda

In [5]:
!python filter_in_video_mp4.py

Procesando video...
Extrayendo audio...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab -

# 6) Descargar el video generado aplicando filtros con pycuda

In [6]:
from google.colab import files
FINAL_VIDEO_MP4_GPU = "video_with_filter_gpu.mp4"

files.download(FINAL_VIDEO_MP4_GPU)
print("Se ha descargado en la PC el archivo de video resultante ")




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Se ha descargado en la PC el archivo de video resultante 


# 7) usar el profiling de nvprof para calcular medir los tiempos de ejecucion

In [ ]:
!nvprof python filter_in_video_mp4.py

==8944== NVPROF is profiling process 8944, command: python3 filter_in_video_mp4.py
Procesando video...
Extrayendo audio...
==8969== Warning: Child processes are not profiled. Use option --profile-child-processes to profile them.

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbit